Multigrid and Multilevel Methods
===

Multigrid (MG) and Multilevel (ML) algorithms provide preconditioners with optimal condition numbers $\kappa (C^{-1} A) = O(1)$, and optimal computational complexity $O(N)$.

They can be seen as extension of the two level overlapping domain decomposition method to more levels.

* Ulrich Trottenberg, Cornelius W. Oosterlee, Anton Schuller: Multigrid, Academic Press, 2001
* Wolfgang Hackbusch: Multi-Grid Methods and Applications, Springer, 1985

In short, both methods are sub-space correction methods where the space splitting is defined by all basis functions together from a hierarchy of grids. Multilevel methods are additive Schwarz methods, while Multigrid methods are the multiplicative Schwarz methods. Both can be implemented efficiently by recursive algorithms. 

We will present different theories for their analysis, one is based on sub-space decomposition using the ASM lemma, the other one uses the smoothing-and-approximation properties.

Multilevel preconditioner
---

We have a sequence of hierarchically refined meshes, which lead to a sequence of nested finite element spaces

$$
V_0 \subset V_1 \subset \ldots V_L
$$

of dimension $N_l = \operatorname{dim} V_l, l = 0 \ldots L$.
We think of mesh-sizes $h_l = 2^{-l}$, leading to finite element space dimensions $2^{dl}$, where $d$ is the spatial dimension.

We have prolongation matrices 

$$
P_l \in {\mathbb R}^{N_l \times N_{l-1}}.
$$

If $v_{l-1}$ is a finite element function in $V_{l-1}$ represented by the coefficient vector $\underline v_{l-1}$. Then
$\underline v_l = P_l \underline v_{l-1}$ is the coefficient vector of the same function represented by the basis functions of $V_l$.

If $A_l$ and $A_{l-1}$ are discretization matrices by a Galerkin method, then there holds

$$
A_{l-1} = P_l^T A_l P_l
$$

Let $D_l = \operatorname{diag} A_l$ be the Jacobi preconditioner (or some similar, cheap and local preconditioner).


**2-level method**

A 2-level preconditioner involving levels $l-1$ and level $l$ is

$$
C_{2L}^{-1} = D_l^{-1} + P_l A_{l-1}^{-1} P_l^T
$$

By the ASM - Lemma we have the tools to analyze that this preconditioner has optimal condition number. However, a direct inversion of the matrix $A_{l-1}$ is, up to a constant factor, as expensive as the inversion of $A_l$. The multilevel method is to replace the inverses recursively by a multilevel preconditioner. On the coarsest level we invert the matrix $A_0$:

$$
C_{ML,0}^{-1}  =  A_0^{-1} 
$$
$$
C_{ML,l}^{-1}  =  D_l^{-1} + P_l C_{ML,l-1}^{-1} P_l^T \qquad \text{for} \; l = 1, \ldots L
$$

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=3, dirichlet=".*", autoupdate=True)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)

In [2]:
class MLPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())
        
    def Mult (self, x, y):
        if self.level == 0:
            y.data = self.localpre * x
            return
        hx = x.CreateVector(copy=True)
        self.fes.Prolongation().Restrict(self.level, hx)
        cdofs = self.fes.Prolongation().LevelDofs(self.level-1)
        y[cdofs] = self.coarsepre * hx[cdofs] 
        self.fes.Prolongation().Prolongate(self.level, y)
        y += self.localpre * x

    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)

With operator notation we can define the multilevel preconditioner also as follows:

In [3]:
def MLPreconditioner2(fes, level, mat, coarsepre):
    prol = fes.Prolongation().Operator(level)
    localpre = mat.CreateSmoother(fes.FreeDofs())
    return localpre + prol @ coarsepre @ prol.T

In [4]:
a.Assemble()
pre = a.mat.Inverse(fes.FreeDofs())

In [5]:
for l in range(7):
    mesh.Refine()
    print ("ndof = ", fes.ndof)
    a.Assemble()
    pre = MLPreconditioner(fes,l+1, a.mat, pre)
    
    #lam = EigenValues_Preconditioner(a.mat, pre)
    #print ("lammin, lammax=", lam[0], lam[-1], \
    #        "kappa=", lam[-1]/lam[0])



ndof =  469
ndof =  1801
ndof =  7057
ndof =  27937
ndof =  111169
ndof =  443521
ndof =  1771777


In [9]:
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver
with TaskManager():
    #pre2 = a.mat.CreateSmoother(fes.FreeDofs())
    inv = CGSolver(mat=a.mat, pre=pre, printrates="\r", maxiter=10000)
    gfu.vec.data = inv * f.vec

CG converged in 109 iterations to residual 1.8663624289726646e-13


In [7]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [8]:
print("dofs" , fes.ndof)

dofs 1771777
